In [ ]:


import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection

db = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='arnav', query={'read_default_file': '~/.my.cnf', 'charset':'utf8mb4'})
engine = sqlalchemy.create_engine(db)

query = "SELECT * FROM feat$cat_met_a30_2000_cp_w$facebook_2020$message_id$1gra"
df = pd.read_sql_query(query, engine)

In [5]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection

db = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='dlatk_lexica', query={'read_default_file': '~/.my.cnf', 'charset':'utf8mb4'})
engine = sqlalchemy.create_engine(db)

# Using the inspector to get table information
inspector = reflection.Inspector.from_engine(engine)

# Retrieve the table names
tables = inspector.get_table_names()

query = "SELECT * FROM met_a30_2000_cp"
outcomes = pd.read_sql_query(query, engine)

db2 = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='arnav', query={'read_default_file': '~/.my.cnf', 'charset':'utf8mb4'})
engine2 = sqlalchemy.create_engine(db2)

outcomes.to_sql('met_a30_2000_cp', engine2, if_exists='replace', index=False, chunksize=500)

In [26]:
import pandas as pd
import sqlalchemy

db = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='loneliness', query={'read_default_file': '~/.my.cnf', 'charset':'utf8mb4'})
engine = sqlalchemy.create_engine(db)

loneliness_query = "SELECT * FROM loneliness_outcomes"
loneliness_outcomes = pd.read_sql_query(loneliness_query, engine)

demographic_query = "SELECT * FROM loneliness_outcomes_withdemographics"
demographic_outcomes = pd.read_sql_query(demographic_query, engine)

user_query = "SELECT * FROM user_outcomes"
user_outcomes = pd.read_sql_query(user_query, engine)
user_outcomes = user_outcomes.rename(columns = {'PHQ_score': 'depression', 'PSS_score': 'stress'})

df = pd.read_sql_query("SELECT * FROM msgs_fb1_dedup", engine)

In [27]:
outcomes = loneliness_outcomes[['user_id', 'age', 'sex', 'marital_status', 'income', 'education']].merge(demographic_outcomes[['user_id', 'total_loneliness_score']], how='inner', on='user_id')
outcomes = outcomes.merge(user_outcomes[['user_id', 'depression', 'stress', 'Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional_Stability', 'Openness_to_newexp', 'UCLA_3item_sum']], how='inner', on='user_id')

In [28]:
remove_marital_status = ['Widowed', 'Separated', 'Partner', 'Prefer not to answer']
outcomes = outcomes[outcomes['marital_status'].isin(remove_marital_status) == False].reset_index(drop=True)

remove_income = ['Less than $10,000', 'Prefer not to say']
outcomes = outcomes[outcomes['income'].isin(remove_income) == False].reset_index(drop=True)

education_income = ["Associate's Degree", "Some college", "Less than high school", "Other", "Prefer not to answer"]
outcomes = outcomes[outcomes['education'].isin(education_income) == False].reset_index(drop=True)

In [29]:
merged = df[['user_id', 'message_id', 'message']].merge(outcomes, how='inner', on='user_id')

counts = merged['user_id'].value_counts().reset_index()
counts.columns = ['user_id', 'message_count']
merged = pd.merge(merged, counts, on = 'user_id', how = 'inner')
merged = merged[merged['message_count'] > 100].reset_index(drop=True)

counts = merged['user_id'].value_counts().reset_index()
total = 0
curr = 0
user_id_set = set()

for i in range(len(counts)):
    user_id_set.add(counts.iloc[i]['index'])
    total += counts.iloc[i]['user_id']
    if total > 1000000:
        break

merged = merged[merged['user_id'].isin(list(user_id_set))].reset_index(drop=True)
merged = merged.sample(1000000, random_state = 42).reset_index(drop=True).dropna()

In [30]:
# merged.to_csv('/sandata/arnav/facebook.csv')
merged.to_csv('facebook.csv')

In [31]:
merged.to_csv('/sandata/arnav/facebook.csv')

In [20]:
db = sqlalchemy.engine.url.URL(drivername='mysql', host='127.0.0.1', database='arnav', query={'read_default_file': '~/.my.cnf', 'charset':'utf8mb4'})
engine = sqlalchemy.create_engine(db)

merged = merged[['message']].reset_index()
merged.columns = ['message_id', 'message']

merged.to_sql('facebook', engine, if_exists='replace', index=False, chunksize=500)